In [37]:
import os
import json
import numpy as np
import datetime
import pandas as pd

In [3]:
path = '/Users/ncr5341/Documents/Coursework/DataPipeline/yelp_dataset/'

In [4]:
business_df = pd.read_json(path+'yelp_academic_dataset_business.json', lines=True)

In [5]:
review = open(path + 'yelp_academic_dataset_review.json').readlines()

In [6]:
reviews = []
for i in review:
    reviews.append(json.loads(i))
review_df = pd.DataFrame.from_records(reviews)

In [12]:
review_df['new_date'] = pd.to_datetime(review_df.date, errors='coerce')

In [24]:
review_newest = review_df[['business_id', 'new_date']].groupby(['business_id']).max()

In [25]:
review_oldest = review_df[['business_id', 'new_date']].groupby(['business_id']).min()

In [79]:
review_duration = review_oldest.join(review_newest, on='business_id', lsuffix='_old', rsuffix='_new')

In [80]:
review_duration['duration'] = review_duration.new_date_new - review_duration.new_date_old

In [81]:
business_tokeep = review_duration[review_duration['duration']>datetime.timedelta(days=365*2)]

In [83]:
len(business_tokeep)

141613

In [85]:
pan_start = datetime.datetime.fromisoformat('2020-03-01')
business_prePandemic = business_tokeep[business_tokeep["new_date_new"] < pan_start]

In [86]:
len(business_prePandemic)

65875

In [91]:
#business_set = business_tokeep.where(business_tokeep['new_date_new'] > pan_start, pan_start)

KeyboardInterrupt: 

In [88]:
business_tokeep.to_pickle('business_tokeep.pkl')
business_prePandemic.to_pickle('business_prePandemic.pkl')

In [99]:
test['new_date_new'].where(test['new_date_new'] > pan_start, pan_start)

business_id
--0DF12EMHYI8XIgoFha6A   2020-12-21 16:40:17
--0r8K_AQ4FZfLsX3ZYRDA   2020-04-15 13:33:08
--164t1nclzzmca7eDiJMw   2020-03-01 00:00:00
--2aF9NhXnNVpDV0KS3xBQ   2021-01-01 14:39:44
--2mEJ63SC_8_08_jGgVIg   2020-08-22 05:04:13
Name: new_date_new, dtype: datetime64[ns]

In [165]:
len(business_tokeep)

141613

In [149]:
business_pan_eligible = business_tokeep.copy()

In [166]:
business_pan_eligible = business_tokeep.assign(pan_duration = pan_start - business_tokeep['new_date_old'])
business_pan_eligible = business_pan_eligible[business_pan_eligible['pan_duration'] > datetime.timedelta(days=365*2)]

In [167]:
business_pan_eligible = business_pan_eligible.assign(duration_toUse = business_pan_eligible[['pan_duration', 'duration']].min(axis=1))

In [169]:
business_pan_eligible.to_pickle('business_pan_eligible.pkl')

In [168]:
len(business_pan_eligible)

138023

In [170]:
business_pan_eligible

,new_date_old,new_date_new,duration,pan_duration,duration_toUse
business_id,,,,,
--0DF12EMHYI8XIgoFha6A,2014-11-25 20:35:03,2020-12-21 16:40:17,2217 days 20:05:14,1922 days 03:24:57,1922 days 03:24:57
--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:15:48,2020-04-15 13:33:08,954 days 20:17:20,909 days 06:44:12,909 days 06:44:12
--164t1nclzzmca7eDiJMw,2009-03-17 01:53:31,2014-06-09 21:27:05,1910 days 19:33:34,4001 days 22:06:29,1910 days 19:33:34
--2aF9NhXnNVpDV0KS3xBQ,2008-08-08 17:13:26,2021-01-01 14:39:44,4528 days 21:26:18,4222 days 06:46:34,4222 days 06:46:34
--2mEJ63SC_8_08_jGgVIg,2015-09-25 00:41:57,2020-08-22 05:04:13,1793 days 04:22:16,1618 days 23:18:03,1618 days 23:18:03
...,...,...,...,...,...
zzngBb9MoGpkFe8tfowvvA,2010-01-13 03:21:35,2020-10-06 15:42:57,3919 days 12:21:22,3699 days 20:38:25,3699 days 20:38:25
zzqq8J7Pibxod1YcknlkWA,2014-08-10 05:00:35,2017-10-23 15:34:03,1170 days 10:33:28,2029 days 18:59:25,1170 days 10:33:28
zzwK-TJsCJX5wZrdtKemPg,2007-10-09 16:13:47,2016-08-02 16:31:57,3220 days 00:18:10,4526 days 07:46:13,3220 days 00:18:10
